# Annotated in Python
> Annotated in Python is a special typing tool that allows us to add extra information(metadata) to a variable, function parameter, or return type

* **Note**
    * These metadata does not affect on how our code runs. It is only used by tools, libraries or frameworks(for Example, LangChain, FastAPI, Pydantic) to understand code better.
    * Why use `Annotated`
        1. To **attach validation rules**.
        2. To **add documentation**.
        3. To **guide external tools**.
        4. To **improve static analysis**.
* **Basic Syntax**
```python
from typing import Annotated

Annotated[type, metadata1, metadata2,...]
```
* type: The original type (like `int`, `str`, `list`).
* metadata: Any extra information(can be any object, string, classes, numbers, etc)

In [ ]:
from typing import Annotated

positive_int = Annotated[int, "Must be positive"]


def process_number(num: positive_int) -> None:
    print(f"Processing number: {num}")


process_number(5)

Processing number: 5


* `num` is an int, but we are adding extra info `"must be positive"`.
* Python itself does not enforce(validate) the positivity --- but other tools(like Pydantic) can read this annoation and apply the rule.

* **More Practical Example (with classes)**
    * Here you even attach custom classes as metadata:

In [41]:
from typing import Annotated

from annotated_types import MaxLen


class MinLength:
    def __init__(self, length: int):
        self.length = length

    def describe(self):
        if self.length < 3:
            return f"Not Allowed {self.length}"
        else:
            return f"Allowed {self.length}"


class MaxLength:
    def __init__(self, length: int):
        self.length = length

    def describe(self):
        if self.length > 50:
            return f"Not Allowed {self.length}"
        else:
            return f"Allowed {self.length}"


Name = Annotated[str, MinLength(3), MaxLen(50)]


def greet(name: Name) -> None:
    print(f"Hello, {name}")


greet("alice")

# Here also function itself does not check this -- but external tools (like validation libraries) can use this information to enforce it.

Hello, alice


## How to Access `Annotated` Metadata
* If you want to read the metadata manually, you can use the `get_type_hints()` function

In [17]:
from typing import get_type_hints

hints = get_type_hints(greet, include_extras=True)

hints

{'name': typing.Annotated[str, <__main__.MinLength object at 0x000002164CF05FD0>, MaxLen(max_length=50)],
 'return': NoneType}

1. Use `typing.get_type_hints()` to retrieve the type hints.
2. Use `typing.get_origin()` and `typing.get_args()` to **break** down the `Annotated` structure.

In [19]:
from typing import get_type_hints

# include_extras = True to keep the metadata
hints = get_type_hints(greet, include_extras=True)
hints

{'name': typing.Annotated[str, <__main__.MinLength object at 0x000002164CF05FD0>, MaxLen(max_length=50)],
 'return': NoneType}

In [33]:
from typing import Annotated, get_type_hints, get_origin, get_args

# Get the type hints
hints = get_type_hints(greet, include_extras=True)

# Access the annotation for 'name'
name_annotation = hints['name']

# Break down the annotation
origin = get_origin(name_annotation)
args = get_args(name_annotation)

print(f"origin: {origin}")
print(f"Base type: {args[0]}")
print(f"Metadata: {args[1:]}")

origin: typing.Annotated
Base type: <class 'str'>
Metadata: (<__main__.MinLength object at 0x000002164CF05FD0>, MaxLen(max_length=50))


## Let's Validate it.

In [43]:
from typing import get_type_hints, get_origin, get_args

hints = get_type_hints(greet, include_extras=True)
name_annotation = hints['name']

origin = get_origin(name_annotation)
args = get_args(name_annotation)

metadata = args[1:]

for meta in metadata:
    if isinstance(meta, MaxLength) or isinstance(meta, MinLength):
        print(meta.describe())

dict_items([('name', typing.Annotated[str, <__main__.MinLength object at 0x000002164DAF8830>, MaxLen(max_length=50)]), ('return', <class 'NoneType'>)])
Allowed 3


## ⚠️ Building Small Validator.

In [44]:
from typing import Annotated, get_type_hints, get_origin, get_args

# Step 1: Define metadata classes


class MinValue:
    def __init__(self, value: int):
        self.value = value

    def describe(self):
        return f"Minimum allowed value is {self.value}"


# Step 2: Define an annotated type
PositiveInt = Annotated[int, MinValue(10)]

# Step 3: Define a function using the annotated type


def process_number(x: PositiveInt) -> None:
    print(f"Processing number: {x}")

# Step 4: Create the validator function


def validate_function_input(func, *args, **kwargs):
    hints = get_type_hints(func, include_extras=True)

    for param_name, param_type in hints.items():
        value = kwargs.get(param_name)
        if value is None:
            continue  # Skip if not passed

        # Check if it is an Annotated type
        if get_origin(param_type) is Annotated:
            base_type, *metadatas = get_args(param_type)

            # Check all metadata
            for metadata in metadatas:
                if isinstance(metadata, MinValue):
                    if value < metadata.value:
                        raise ValueError(
                            f"Validation failed: {param_name} must be >= {metadata.value}, got {value}"
                        )

    # If all validations pass
    return func(*args, **kwargs)


# Step 5: Test the validation
try:
    validate_function_input(process_number, x=5)  # Should raise an error
except ValueError as e:
    print(e)

try:
    validate_function_input(process_number, x=15)  # Should succeed
except ValueError as e:
    print(e)

Validation failed: x must be >= 10, got 5
Processing number: 15
